In [1]:
# calibrate_and_stability.py
import numpy as np
import pandas as pd
import os
from itertools import combinations
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, f1_score, precision_recall_fscore_support,
    roc_auc_score, precision_recall_curve
)
from scipy.stats import spearmanr

# ------------- CONFIG --------------
FEATURES_CSV = "features_stress.csv"          # input features produced earlier
RESULTS_CSV = "anomaly_results_stress.csv"    # optional; used for explanation fields (not required)
OUT_DIR = "calibration_output"
SEEDS = [0, 1, 2, 3, 5, 7, 11, 13, 17, 19]    # multiple seeds to test stability
N_ESTIMATORS = 200
# If true labels exist in features CSV, the script will use them; otherwise it runs unsupervised-only checks.
# If you know expected anomaly fraction, set this; otherwise we'll infer from labels if present.
CONTAMINATION = None   # None -> infer from y_true if available; otherwise set a sensible default like 0.02
TOP_K_RULE = None      # None -> compute K from contamination * n_samples
# ------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)

def load_features(path):
    assert os.path.exists(path), f"{path} not found"
    df = pd.read_csv(path, index_col=0)
    # if is_attack present use it as ground truth
    y_true = df["is_attack"].astype(int) if "is_attack" in df.columns else None
    X = df.drop(columns=["is_attack"]) if "is_attack" in df.columns else df.copy()
    return X, y_true

def train_iforest_get_scores(X, seed, contamination):
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    model = IsolationForest(n_estimators=N_ESTIMATORS, contamination=contamination, random_state=seed)
    model.fit(Xs)
    # sklearn decision_function: larger => more normal. Convert so larger => more anomalous
    scores = -model.decision_function(Xs)   # higher = more anomalous
    preds = model.predict(Xs)               # -1 anomaly, 1 normal
    y_pred = np.where(preds == -1, 1, 0)
    return scores, y_pred, model, scaler

def percentile_from_scores(scores):
    # compute percentile rank so final score is in [0,1] with 1 = most anomalous
    ranks = scores.argsort().argsort()  # 0..N-1
    # we want percentile = rank / (N-1)
    if len(scores) <= 1:
        return np.ones_like(scores)
    perc = ranks / (len(scores) - 1)
    return perc

def jaccard_topk(setA, setB):
    if len(setA) == 0 and len(setB) == 0:
        return 1.0
    inter = len(setA.intersection(setB))
    union = len(setA.union(setB))
    return inter / union if union > 0 else 0.0

def main():
    X, y_true = load_features(FEATURES_CSV)
    n = X.shape[0]
    print(f"Loaded features: {X.shape[0]} users, {X.shape[1]} features")
    # determine contamination
    global CONTAMINATION, TOP_K_RULE
    if CONTAMINATION is None:
        if y_true is not None:
            CONTAMINATION = float(max(0.001, y_true.mean()))  # use empirical rate
            print(f"Inferred contamination from labels: {CONTAMINATION:.4f}")
        else:
            CONTAMINATION = 0.02
            print(f"No labels found; using default contamination = {CONTAMINATION:.4f}")
    if TOP_K_RULE is None:
        TOP_K_RULE = max(1, int(round(CONTAMINATION * n)))
        print(f"Top-K (for overlap) set to {TOP_K_RULE} (contamination*N)")

    seed_results = {}
    all_percentiles = {}
    all_scores = {}

    # Train with multiple seeds and collect scores & predictions
    for seed in SEEDS:
        scores, y_pred, model, scaler = train_iforest_get_scores(X, seed, CONTAMINATION)
        perc = percentile_from_scores(scores)  # 0..1
        # we want 1 = most anomalous; currently perc low -> less anomalous because rank ascending. Fix:
        # Because scores higher => more anomalous (we used -decision_function), ranks ascending -> small rank means least anomalous.
        # Let's convert so perc_anom = 1 - perc
        perc_anom = 1.0 - perc
        seed_results[seed] = {"scores": scores, "percentile": perc_anom, "y_pred": y_pred}
        all_percentiles[seed] = perc_anom
        all_scores[seed] = scores
        # save per-seed CSV
        df_seed = pd.DataFrame({
            "user": X.index,
            "anomaly_score_raw": scores,
            "anomaly_percentile": perc_anom,
            "predicted": y_pred
        }).set_index("user")
        df_seed.to_csv(os.path.join(OUT_DIR, f"seed_{seed}_scores.csv"))

    # Stability analysis: Spearman rank corr between seeds, and top-K Jaccard
    pairs = list(combinations(SEEDS, 2))
    spearman_vals = []
    jaccard_vals = []
    for a, b in pairs:
        sa = all_scores[a]
        sb = all_scores[b]
        # Spearman between raw anomaly scores
        rho, p = spearmanr(sa, sb)
        spearman_vals.append(rho if not np.isnan(rho) else 0.0)
        # top-K sets
        topA = set(X.index[np.argsort(-sa)][:TOP_K_RULE])  # sort descending -> most anomalous first
        topB = set(X.index[np.argsort(-sb)][:TOP_K_RULE])
        jacc = jaccard_topk(topA, topB)
        jaccard_vals.append(jacc)
    mean_spearman = np.mean(spearman_vals) if spearman_vals else 1.0
    mean_jaccard = np.mean(jaccard_vals) if jaccard_vals else 1.0

    # Ensemble aggregation: average percentiles across seeds
    perc_matrix = np.vstack([all_percentiles[s] for s in SEEDS]).T  # n x seeds
    ensemble_percentile = perc_matrix.mean(axis=1)
    ensemble_scores_df = pd.DataFrame({
        "user": X.index,
        "ensemble_percentile": ensemble_percentile
    }).set_index("user")
    ensemble_scores_df.to_csv(os.path.join(OUT_DIR, "ensemble_percentile.csv"))

    # If labels available, evaluate varying percentile thresholds to choose an operating point
    best_thresh = None
    best_f1 = -1
    threshold_grid = np.linspace(0.01, 0.99, 99)
    metrics = []
    if y_true is not None:
        y = y_true.loc[X.index].astype(int).values
        # compute AUC on ensemble_percentile
        try:
            auc = roc_auc_score(y, ensemble_percentile)
        except Exception:
            auc = None
        for t in threshold_grid:
            yhat = (ensemble_percentile >= t).astype(int)
            p, r, f, _ = precision_recall_fscore_support(y, yhat, average='binary', zero_division=0)
            metrics.append((t, p, r, f))
            if f > best_f1:
                best_f1 = f
                best_thresh = t
        # save calibration table
        calib_df = pd.DataFrame(metrics, columns=["threshold", "precision", "recall", "f1"]).set_index("threshold")
        calib_df.to_csv(os.path.join(OUT_DIR, "threshold_calibration.csv"))
    else:
        auc = None

    # Save summary CSV with per-user info from ensemble and per-seed raw scores
    summary = X.copy()
    summary.index = X.index
    for seed in SEEDS:
        summary[f"score_s{seed}"] = all_scores[seed]
        summary[f"perc_s{seed}"] = all_percentiles[seed]
        summary[f"pred_s{seed}"] = seed_results[seed]["y_pred"]
    summary["ensemble_percentile"] = ensemble_percentile
    if y_true is not None:
        summary["true_attack"] = y_true
    summary.to_csv(os.path.join(OUT_DIR, "stability_summary.csv"))

    # Print short human summary and recommendations
    print("=== Stability & Calibration Summary ===")
    print(f"Seeds used: {SEEDS}")
    print(f"Samples: {n}, features: {X.shape[1]}")
    print(f"Contamination used: {CONTAMINATION:.4f}, Top-K for overlap: {TOP_K_RULE}")
    print(f"Mean Spearman rank correlation across seeds: {mean_spearman:.4f}")
    print(f"Mean Top-{TOP_K_RULE} Jaccard overlap across seeds: {mean_jaccard:.4f}")
    if auc is not None:
        print(f"Ensemble ROC-AUC (using true labels): {auc:.4f}")
    if best_thresh is not None:
        # compute metrics at best threshold
        best_row = calib_df.loc[best_thresh]
        print(f"Best F1 threshold (ensemble percentile): {best_thresh:.3f} -> precision={best_row['precision']:.3f}, recall={best_row['recall']:.3f}, f1={best_row['f1']:.3f}")
        print(f"Calibration table saved: {os.path.join(OUT_DIR, 'threshold_calibration.csv')}")
    print(f"Per-user stability summary saved: {os.path.join(OUT_DIR, 'stability_summary.csv')}")
    print(f"Per-seed CSVs and ensemble file saved in {OUT_DIR}/")
    print("\nRecommendations:")
    if mean_spearman < 0.7:
        print("- Ranking is unstable (Spearman < 0.7). Consider: increase n_estimators, train with more seeds and ensemble scores, or refine features.")
    else:
        print("- Ranking is relatively stable across seeds.")
    if mean_jaccard < 0.5:
        print("- Top-K overlap low. Consider: ensemble over seeds (we produced ensemble_percentile.csv), or increase signal via features.")
    if best_thresh is not None and best_f1 < 0.4:
        print("- Model F1 is low at best threshold. Consider: adding features, ensembling with Autoencoder, or adding rule signals.")
    print("\nNext recommended actions:")
    print("1) Inspect 'stability_summary.csv' and 'ensemble_percentile.csv' to see which users are top-ranked.")
    print("2) Use ensemble_percentile as the canonical anomaly score for downstream risk aggregation.")
    print("3) If ranking unstable, run Autoencoder as an orthogonal detector and ensemble its percentiles with IF (average).")
    print("4) Use the calibration table to pick an operational percentile threshold for alerts (or to map percentile -> risk bands).")

if __name__ == "__main__":
    main()


Loaded features: 500 users, 9 features
Inferred contamination from labels: 0.0200
Top-K (for overlap) set to 10 (contamination*N)
=== Stability & Calibration Summary ===
Seeds used: [0, 1, 2, 3, 5, 7, 11, 13, 17, 19]
Samples: 500, features: 9
Contamination used: 0.0200, Top-K for overlap: 10
Mean Spearman rank correlation across seeds: 0.9828
Mean Top-10 Jaccard overlap across seeds: 0.8007
Ensemble ROC-AUC (using true labels): 0.0122
Best F1 threshold (ensemble percentile): 0.010 -> precision=0.010, recall=0.500, f1=0.020
Calibration table saved: calibration_output\threshold_calibration.csv
Per-user stability summary saved: calibration_output\stability_summary.csv
Per-seed CSVs and ensemble file saved in calibration_output/

Recommendations:
- Ranking is relatively stable across seeds.
- Model F1 is low at best threshold. Consider: adding features, ensembling with Autoencoder, or adding rule signals.

Next recommended actions:
1) Inspect 'stability_summary.csv' and 'ensemble_percentil